# Radar Plan Position Indicator (PPI) in 3D

This example is adapted from [xradar's PPI plot example](https://docs.openradarscience.org/projects/xradar/en/stable/notebooks/plot-ppi.html) to visualize radar sweeps as a 3D volume using pyvista-xarray.

Radar data naturally lives in polar coordinates (range, azimuth, elevation). After georeferencing with xradar, the x/y/z coordinates form a **curvilinear grid** — each point has a unique 3D position that doesn't lie on regular grid lines. This is exactly the use case for `pyvista.StructuredGrid`, which pyvista-xarray creates automatically when it detects multi-dimensional coordinates.

In [ ]:
from open_radar_data import DATASETS
import pyvista as pv
import xarray as xr
import xradar as xd

import pvxarray  # noqa: F401

pv.set_jupyter_backend("server")

## Load and Georeference Radar Data

Fetch a CfRadial1 radar file and open it as an xarray DataTree. The `georeference()` call computes Cartesian x/y/z coordinates from the native polar coordinates:

In [ ]:
filename = DATASETS.fetch("cfrad.20080604_002217_000_SPOL_v36_SUR.nc")
radar = xd.io.open_cfradial1_datatree(filename)
radar = radar.xradar.georeference()  # Compute x/y/z from polar coords
list(radar.children)

## Stack Sweeps into a 3D Volume

Each sweep is a 2D surface at a different elevation angle. Concatenating them creates a 3D data volume that reveals the vertical structure of storms:

In [ ]:
ds_list = [
    radar[key].ds.drop_duplicates(dim="azimuth") for key in list(radar.children) if "sweep" in key
]
ds = xr.concat(ds_list, dim="sweep")
ds

## Create a StructuredGrid Mesh

Since x, y, and z are all 3D arrays (one value per point), pyvista-xarray automatically creates a `StructuredGrid` — a curvilinear mesh that follows the radar's conical scan geometry:

In [ ]:
mesh = ds["DBZ"].pyvista.mesh(x="x", y="y", z="z")
mesh

## Visualize the Reflectivity Volume

A direct plot shows the full 3D extent of the radar scan. The reflectivity values (dBZ) indicate precipitation intensity:

In [ ]:
mesh.plot(clim=(-20, 60))

## Pseudo-Volume Rendering with Contours

Extract isosurfaces (contours) of reflectivity to create a pseudo-volume rendering. This highlights the 3D structure of precipitation cores within the storm:

In [ ]:
pl = pv.Plotter()
pl.add_mesh(mesh.contour(), opacity=0.9, clim=(-20, 60), lighting=False)
pl.show(jupyter_backend="static")